In [1]:
from azureml.core import Workspace, Experiment

ws = Workspace.from_config()
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

Workspace name: quick-starts-ws-229711
Azure region: westeurope
Subscription id: 6971f5ac-8af1-446e-8034-05acea24681f
Resource group: aml-quickstarts-229711


In [3]:
from azureml.core.compute import ComputeTarget, AmlCompute

cluster_name = 'vm-proj'

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.


try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
except:
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', 
                                                           max_nodes=4)
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True)


Running


In [7]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling({"C":  uniform(0, 1), "max_iter": choice(25, 50, 100, 150, 200, 250)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=2, slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig('.', 'train.py', compute_target=compute_target)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                     hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=20,
                                     max_concurrent_runs=4)

In [8]:
# Submit your hyperdrive run to the experiment and show run details with the widget.
hyperdrive_run = exp.submit(config=hyperdrive_config)
RunDetails(hyperdrive_run).show()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_d7aeea36-1aac-401e-8c1a-ccc27c45ebad
Web View: https://ml.azure.com/runs/HD_d7aeea36-1aac-401e-8c1a-ccc27c45ebad?wsid=/subscriptions/6971f5ac-8af1-446e-8034-05acea24681f/resourcegroups/aml-quickstarts-229711/workspaces/quick-starts-ws-229711&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-04-04T14:12:27.863392][GENERATOR][INFO]Trying to sample '4' jobs from the hyperparameter space
[2023-04-04T14:12:28.6225944Z][SCHEDULER][INFO]Scheduling job, id='HD_d7aeea36-1aac-401e-8c1a-ccc27c45ebad_0' 
[2023-04-04T14:12:28.6252345Z][SCHEDULER][INFO]Scheduling job, id='HD_d7aeea36-1aac-401e-8c1a-ccc27c45ebad_1' 
[2023-04-04T14:12:28.7591187Z][SCHEDULER][INFO]Scheduling job, id='HD_d7aeea36-1aac-401e-8c1a-ccc27c45ebad_2' 
[2023-04-04T14:12:28.804366][GENERATOR][INFO]Successfully sampled '4' jobs, they will soon be submitted to the execution target.
[2023-04-04T14:12:28.9138221Z][SCHEDULER][INFO]Scheduling job, id='HD_d7aeea36-1aac-401e-8c1a-ccc27c45eb

ActivityFailedException: ActivityFailedException:
	Message: Activity Failed:
{
    "error": {
        "code": "UserError",
        "message": "Execution failed. User process '/azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/bin/python' exited with status code 1. Please check log file 'user_logs/std_log.txt' for error details. Error: Traceback (most recent call last):\n  File \"train.py\", line 1, in <module>\n    from sklearn.linear_model import LogisticRegression\nModuleNotFoundError: No module named 'sklearn'\n\n Marking the experiment as failed because initial child jobs have failed due to user error",
        "messageParameters": {},
        "details": []
    },
    "time": "0001-01-01T00:00:00.000Z"
}
	InnerException None
	ErrorResponse 
{
    "error": {
        "message": "Activity Failed:\n{\n    \"error\": {\n        \"code\": \"UserError\",\n        \"message\": \"Execution failed. User process '/azureml-envs/azureml_f3f7e6c5fb83d94df23933000bf02da3/bin/python' exited wi

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyperdrive_run.get_best_run_by_primary_metric()
model = best_run.register_model(model_name='best_reg_bank')
filename = "best_logreg_bank.pkl"
joblib.dump(value=model, filename=filename)

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

ds = TabularDatasetFactory.from_delimited_files(
    path="https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv", 
)

In [ ]:
from train import clean_data
from sklearn.model_selection import train_test_split

# Use the clean_data function to clean your data.

x, y = clean_data(ds)
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    primary_metric='AUC_weighted',
    training_data=x_train,
    label_column_name='y',
    n_cross_validations=5)

In [ ]:
# Submit your automl run

run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.

best_run = run.get_best_child()
model_name = best_run.properties["model_name"]
model = run.register_model(model_name=model_name)
filename = 'best_automl_bank.pkl'
joblib.dump(value=model, filename=filename)